In [4]:
import requests
from bs4 import BeautifulSoup

def get_ship_type_from_mmsi(mmsi):
    url = f"https://www.vesselfinder.com/vessels/details/{mmsi}"
    response = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
    soup = BeautifulSoup(response.text, "html.parser")

    ais_type_cell = soup.find("td", string="AIS Type")

    if ais_type_cell:
        ship_type = ais_type_cell.find_next_sibling("td", class_="v3").get_text(strip=True)
        return ship_type
    else:
        return "Ship type not found"
test_mmsi = "0"
ship_type = get_ship_type_from_mmsi(test_mmsi)
print(f"Ship type for MMSI {test_mmsi}: {ship_type}")

KeyboardInterrupt: 

In [5]:
import pandas as pd
import time

def get_ship_types_from_file(file_path):

    df = pd.read_csv(file_path)

    unique_mmsi = df['MMSI'].unique()

    # only test
    mmsi_type_dict = {}

    for mmsi in unique_mmsi:
        ship_type = get_ship_type_from_mmsi(mmsi)
        mmsi_type_dict[mmsi] = ship_type
        print(f"MMSI: {mmsi}, Ship Type: {ship_type}")
        time.sleep(3)

    print("Total unique MMSI count:", len(unique_mmsi))
    print("Total unique ship types found:", len(set(mmsi_type_dict.values())))
    return mmsi_type_dict

mmsi_type_dict = get_ship_types_from_file('data/ais_combined.csv')

KeyboardInterrupt: 